### Общий план:
- Заполнить пропуски в данных диагностики
- Агрегировать результаты диагностики по каждому юзеру
- __Выделить фичи из текста__
- __Агрегировать фичи по юзеру__
- __Объединить датасеты__
- Построить матрицу корреляций по полученному датасету
- Наиболее удачные, яркие корреляции рассмотреть подробнее
- Построить бейзлайн, лр/бустинг, для прогнозирования результата диагностики по фичам текста

In [48]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from functools import reduce
from deeppavlov import build_model, configs
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [49]:
# основные данные рефлексий
main_text = pd.read_csv('fixed_reflex_texts.csv')
main_text.shape

(9128, 15)

In [36]:
main_text['question'].unique()

array(['Насколько вероятно, что Вы порекомендуете мероприятие знакомым? Пожалуйста, оцените по шкале от 0 до 10',
       'Перечислите основные этапы деятельности, в которую вы были вовлечены и благодаря этому приобретали новые или развивали существующие компетенции',
       'Чему новому, как вам кажется, вы научились?',
       'Была ли в этом ценность и в чем она выражается?',
       'Поделитесь вашими комментариями или пожеланиями по поводу того, с чем вы познакомились?',
       'Какие вопросы, комментарии  и пожелания у вас есть?',
       'Вы понимаете, зачем пришли на это мероприятие? Оцените по шкале от 1 до 10, где 1 - не совсем, 10 - да, и понимаю, как именно буду использовать полученные знания и навыки в своей дальнейшей деятельности.',
       'Вы понимаете, зачем пришли на это мероприятие? Оцените по шкале от 1 до 10, где 1 - не совсем,  10 -  да, и понимаю, как именно буду использовать полученные знания и навыки в своей дальнейшей деятельности.',
       'Что вы поняли про свой

In [50]:
# возьмем вопросы, для ответов на которые можно оценить тон учащегося
q = [
    'Чему новому, как вам кажется, вы научились?',
 'Была ли в этом ценность и в чем она выражается?',
 'Поделитесь вашими комментариями или пожеланиями по поводу того, с чем вы познакомились?',
 'Какие вопросы, комментарии  и пожелания у вас есть?',
 'Что вы поняли про свой проект по итогам общения с экспертами? К каким действиям это вас побудило?',
 'Что было полезно для вас лично и для команды в целом в трубе экспертов?',
 'А что можно было бы в трубе экспертов улучшить?',
 'Какие вопросы для вас остаются без ответа, чего вам не хватает сейчас?',
 'Если бы вы заранее знали, что труба будет проходить именно так, как бы вы к ней подготовились?',
    'Насколько ясно команда сформулировала, кому и с чем хочет помогать (проблема и пользователи)',
 'Насколько проработанный прототип у команды?',
 'Насколько решение отвечает проблеме?',
 'Уровень амбиций команды (насколько высокую планку поставили)',
 'Насколько хороша презентация команды (дизайн слайдов, структура, подача)',
 'Комментарии и рекомендации команде',
]

In [51]:
main_text = main_text[main_text['question'].isin(q)]
main_text.shape

(5440, 15)

In [52]:
# identify id where answers are not textual
ids_no_sentiment = []

for ind, val in zip(main_text.index, main_text['value']):
    if not isinstance(val, str):
        ids_no_sentiment.append(ind)
        
# slice rows where answers are textual
main_text = main_text[~main_text.index.isin(ids_no_sentiment)]
main_text.shape

(3441, 15)

In [53]:
# доп данные рефлексий
additional_data_text = pd.read_csv('additional_data_text.csv')
additional_data_text.shape

(1256, 2)

In [54]:
additional_data_text.dtypes

untiID                                int64
Чему вы научились в рамках курса?    object
dtype: object

## Sentiment (Анализ тональности текста)

In [96]:
sent_model = build_model(configs.classifiers.rusentiment_elmo_twitter_cnn, download=True)

In [56]:
def do_sentiment(line):
    return sent_model([line])[0]

In [57]:
%%time
# Выделение тональности из основных данных
main_text['sentiment'] = main_text['value'].apply(do_sentiment)
main_text = pd.get_dummies(main_text, columns=['sentiment'])

Wall time: 10min 15s


In [58]:
%%time
# Выделение тональности из доп данных
additional_data_text['sentiment'] = additional_data_text['Чему вы научились в рамках курса?'].apply(do_sentiment)

Wall time: 6min 27s


In [59]:
additional_data_text['sentiment'].value_counts()

neutral     1243
positive      11
speech         2
Name: sentiment, dtype: int64

In [60]:
# encode sentiment as one-hot
additional_data_text = pd.get_dummies(additional_data_text, columns=['sentiment'])

In [61]:
# добавим колонки для сентиментов, найденных в основном датасете рефлексий
additional_data_text['sentiment_skip'] = 0
additional_data_text['sentiment_negative'] = 0

In [62]:
additional_data_text.head(3)

,untiID,Чему вы научились в рамках курса?,sentiment_neutral,sentiment_positive,sentiment_speech,sentiment_skip,sentiment_negative
0,701335,"Оформила страницу в социальной сети, определил...",1,0,0,0,0
1,1033016,До начала курса я немного был знаком с основам...,1,0,0,0,0
2,995121,Очень много новой информации. Научились состав...,1,0,0,0,0


## NER (Именованные сущности)

In [97]:
ner_model = build_model(configs.ner.ner_rus, download=True)

In [64]:
def find_ner(line):
    words, tags = ner_model([line])
    try:
        for w, t in zip(words[0], tags[0]):
            if t != 'O':
                return 1
            else:
                return 0
    except:
        return 0

In [65]:
%%time
main_text['ner'] = main_text['value'].apply(find_ner)
additional_data_text['ner'] = additional_data_text['Чему вы научились в рамках курса?'].apply(find_ner)

Wall time: 16.4 s


In [66]:
additional_data_text['ner'].value_counts()

0    1145
1     111
Name: ner, dtype: int64

In [67]:
additional_data_text.head()

,untiID,Чему вы научились в рамках курса?,sentiment_neutral,sentiment_positive,sentiment_speech,sentiment_skip,sentiment_negative,ner
0,701335,"Оформила страницу в социальной сети, определил...",1,0,0,0,0,0
1,1033016,До начала курса я немного был знаком с основам...,1,0,0,0,0,0
2,995121,Очень много новой информации. Научились состав...,1,0,0,0,0,0
3,674661,"Очень полезный курс, научилась грамотно вести ...",1,0,0,0,0,0
4,969910,научилась очень многому благодаря компетенции ...,0,1,0,0,0,0


### Features: особенности текстов

In [85]:
import string
punctuation = string.punctuation

In [86]:
# длина комментария
main_text["answer_len"] = main_text['value'].apply(len)

# кол-во слов с заглавной буквы
main_text['upper_case_word_count'] = main_text['value'].apply(lambda x: len([wrd for wrd in x.split() if wrd[0].isupper()]))

# кол-во знаков препинания
main_text['punctuation_count'] = main_text['value'].apply(lambda x: len("".join(_ for _ in x if _ in punctuation)))

In [87]:
# длина комментария
additional_data_text["answer_len"] = additional_data_text['Чему вы научились в рамках курса?'].apply(len)

# кол-во слов с заглавной буквы
additional_data_text['upper_case_word_count'] = additional_data_text['Чему вы научились в рамках курса?'].apply(lambda x: len([wrd for wrd in x.split() if wrd[0].isupper()]))

# кол-во знаков препинания
additional_data_text['punctuation_count'] = additional_data_text['Чему вы научились в рамках курса?'].apply(lambda x: len("".join(_ for _ in x if _ in punctuation)))

### Features: использование частей речи

In [69]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [70]:
def count_pos(line, pos):
    """
    Оставляет только верно написанные русские слова и подсчитываем,
    сколько слов определенной частей речи (аргумент pos) в сообщении.
    """
    char_regex = re.compile(r'[^а-яa-z]')
    line = char_regex.sub(' ', line.lower())
    
    chkr_ru = SpellChecker(d_ru)
    chkr_ru.set_text(line)
    mistakes_ru = set([err.word for err in chkr_ru])
    
    tokenized = toktok.tokenize(line)
    count = 0
    for i in tokenized:
        if i not in mistakes_ru:
            p = morph.parse(i)[0]
            if p.tag.POS==pos:
                count+=1
            
    return count

In [71]:
# подсчет существительных
main_text['nouns'] = main_text.apply(lambda x: count_pos(x['value'], 'NOUN'), axis=1)
additional_data_text['nouns'] = additional_data_text.apply(lambda x: count_pos(x['Чему вы научились в рамках курса?'], 
                                                                               'NOUN'), axis=1)

In [72]:
# подсчет глаголов (личная форма)
main_text['verbs'] = main_text.apply(lambda x: count_pos(x['value'], 'VERB'), axis=1)
additional_data_text['verbs'] = additional_data_text.apply(lambda x: count_pos(x['Чему вы научились в рамках курса?'], 
                                                                               'VERB'), axis=1)

In [73]:
# подсчет глаголов (инфинитив)
main_text['inf'] = main_text.apply(lambda x: count_pos(x['value'], 'INFN'), axis=1)
additional_data_text['inf'] = additional_data_text.apply(lambda x: count_pos(x['Чему вы научились в рамках курса?'], 
                                                                               'INFN'), axis=1)

In [74]:
# подсчет прилагательных
main_text['adj'] = main_text.apply(lambda x: count_pos(x['value'], 'ADJF'), axis=1)
additional_data_text['adj'] = additional_data_text.apply(lambda x: count_pos(x['Чему вы научились в рамках курса?'], 
                                                                               'ADJF'), axis=1)

In [75]:
# подсчет наречий
main_text['advb'] = main_text.apply(lambda x: count_pos(x['value'], 'ADVB'), axis=1)
additional_data_text['advb'] = additional_data_text.apply(lambda x: count_pos(x['Чему вы научились в рамках курса?'], 
                                                                               'ADVB'), axis=1)

In [76]:
# подсчет местоимений
main_text['pronoun'] = main_text.apply(lambda x: count_pos(x['value'], 'NPRO'), axis=1)
additional_data_text['pronoun'] = additional_data_text.apply(lambda x: count_pos(x['Чему вы научились в рамках курса?'], 
                                                                               'NPRO'), axis=1)

In [77]:
additional_data_text[['Чему вы научились в рамках курса?','nouns', 'verbs','inf','adj','advb','pronoun']]

,Чему вы научились в рамках курса?,nouns,verbs,inf,adj,advb,pronoun
0,"Оформила страницу в социальной сети, определил...",7,6,1,2,0,0
1,До начала курса я немного был знаком с основам...,16,4,1,2,2,5
2,Очень много новой информации. Научились состав...,4,1,5,3,3,0
3,"Очень полезный курс, научилась грамотно вести ...",8,2,3,5,4,0
4,научилась очень многому благодаря компетенции ...,22,5,2,6,9,2
...,...,...,...,...,...,...,...
1251,Курсы по этой программе весьма интересны и име...,11,2,1,4,2,0
1252,Узнал много инструментов для разработки прилож...,7,2,0,1,1,1
1253,"Да очень, много познавательного интересного уз...",0,1,0,2,2,1
1254,"Есть полезная информация, которую стоит держат...",2,1,1,2,0,1


### Features: кол-во слов с ошибками

Для проверки правописания русских и английских слов спользуем библиотеку `pyenchant`.

In [78]:
import re
import enchant
from enchant.checker import SpellChecker
from nltk.tokenize import ToktokTokenizer

# инициализируем словари
d_ru = enchant.DictWithPWL('ru')
d_eng = enchant.DictWithPWL('ru')

#инициализируем токенайзер
toktok = ToktokTokenizer()

In [79]:
def spell_check(line):
    char_regex = re.compile(r'[^а-яa-z]')
    line = char_regex.sub(' ', line.lower())
    
    chkr_ru = SpellChecker(d_ru)
    chkr_ru.set_text(line)
    mistakes_ru = set([err.word for err in chkr_ru])
    
    chkr_eng = SpellChecker(d_eng)
    chkr_eng.set_text(line)
    mistakes_eng = set([err.word for err in chkr_eng])
    
    tokenized = toktok.tokenize(line)
    num_mistakes=0
    for i in tokenized:
        if i in mistakes_ru and i in mistakes_eng:
            num_mistakes+=1
            
    return num_mistakes

In [80]:
main_text['num_mistakes'] = main_text['value'].apply(spell_check)

In [81]:
additional_data_text['num_mistakes'] = additional_data_text['Чему вы научились в рамках курса?'].apply(spell_check)

In [82]:
main_text['num_mistakes'].value_counts()

0     2750
1      476
2      142
3       43
4       19
6        7
5        2
18       1
8        1
Name: num_mistakes, dtype: int64

In [83]:
additional_data_text['num_mistakes'].value_counts()

0     608
1     276
2     145
3      73
4      43
5      32
6      22
8      11
7      11
10      9
11      5
12      5
9       4
13      4
15      2
22      1
16      1
18      1
20      1
21      1
25      1
Name: num_mistakes, dtype: int64

### Агрегация средних значений выделенных признаков по каждому юзеру

In [88]:
agg_additional_data_text = additional_data_text[['untiID', 'sentiment_negative', 'sentiment_neutral','sentiment_positive', 
                                                 'sentiment_skip','sentiment_speech','ner','answer_len',
                                                 'upper_case_word_count','punctuation_count',
                                                'nouns', 'verbs','inf','adj','advb','pronoun',
                                                 'num_mistakes']].groupby(['untiID'],
                                                      as_index=False).agg(np.mean)

agg_main_text = main_text[['untiID', 'sentiment_negative', 'sentiment_neutral','sentiment_positive', 
                           'sentiment_skip','sentiment_speech','ner','answer_len',
                           'upper_case_word_count','punctuation_count',
                          'nouns', 'verbs','inf','adj','advb','pronoun','num_mistakes']].groupby(['untiID'],
                       as_index=False).agg(np.mean)


In [89]:
agg_additional_data_text.head(1)

,untiID,sentiment_negative,sentiment_neutral,sentiment_positive,sentiment_skip,sentiment_speech,ner,answer_len,upper_case_word_count,punctuation_count,nouns,verbs,inf,adj,advb,pronoun,num_mistakes
0,220,0.0,1,0,0.0,0,0.0,159.0,2.0,4.0,9.0,1.0,1.0,1.0,0.0,1.0,1.0


In [90]:
agg_main_text.head(1)

,untiID,sentiment_negative,sentiment_neutral,sentiment_positive,sentiment_skip,sentiment_speech,ner,answer_len,upper_case_word_count,punctuation_count,nouns,verbs,inf,adj,advb,pronoun,num_mistakes
0,4,0.0,0.972222,0.027778,0.0,0.0,0.583333,1.75,0.027778,0.055556,0.027778,0.0,0.0,0.027778,0.0,0.0,0.027778


In [91]:
# Объединим основные и доп агрегаты
agg = pd.concat([agg_main_text, agg_additional_data_text])
agg.shape

(1634, 17)

### Финальные датасеты
#### Объединим агрегированные текст.фичи и агрегаты диагностик 

In [93]:
df_train = pd.read_csv('diag_large_train.csv')
df_test = pd.read_csv('diag_test_agg.csv')

df_train.shape, df_test.shape

((1451, 327), (127, 681))

In [94]:
train = pd.merge(agg, df_train, on=['untiID'])
test = pd.merge(agg, df_test, on=['untiID'])
test = test[train.columns.tolist()]
train.shape, test.shape

((1417, 343), (110, 343))

In [80]:
train.to_csv('train.csv',index=False)
test.to_csv('test.csv',index=False)